In [4]:
import os
from os.path import expanduser
import socket
import sys


# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "awi-models")
elif running_system[0:1] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"awi-models")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
# IMPORT PY-FESOM MODULES
sys.path.append("codes/pyfesom-master/") # add pyfesom to search path
sys.path.append("codes/modules")
#########################################

computer =  bfn1001
working directory set to /home/hbkoziel/awi-models


In [10]:
import pyfesom as pf
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap
from pylab import *
from load_mesh_data_new import *
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
import colormaps as cmaps
from matplotlib.colors import ListedColormap

In [6]:
# PLOT CONFIGURATION #

# load colormaps #
cmap = cm.RdBu_r
# figure export 
export_plot = True
# figure export definition
dpicnt=150
# choose simulation
runid='Arc12'
# choose date
first_year = 1990
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
months =np.linspace(0,11,12).astype(int)
# choose depth
get3d = True
# load the given biological tracer #
mld_id, par_id   ='mixlay', 'PAR3D_mean'
# export CSV
export_csv = True

########################

# AUTOMATIC DEFINITION OF PATHS
resultpath = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'
savepath    = wd_path+'/figures/'+runid+'/'
meshpath    = wd_path+'/data/mesh/meshArc4.5/'
outputpath = '/scratch/usr/hbkoziel/Arc12/'

# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)

directory /home/hbkoziel/awi-models/figures/Arc12/ already existing


In [7]:
mesh = load_mesh(meshpath, usepickle=True,get3d=True)

The *usepickle = True* and the pickle file (*pickle_mesh*) exists.
 We load the mesh from it.


In [8]:
mesh


FESOM mesh:
path                  = /gfs2/work/hbkvsk12/mesh/meshArc4.5
alpha, beta, gamma    = 50, 15, -90
number of 2d nodes    = 753179
number of 2d elements = 1480268
number of 3d nodes    = 16950142

        

In [9]:
mesh.zlevs = -mesh.zlevs # depth is made negative

In [11]:
PAR  = np.zeros((len(years),len(mesh.x2)))
for mo in range(4,9):
    print mo
    for ye in range(0,len(years)):
     print years[ye]
     if years[ye] < 2000:
        ncfile  = resultpath+runid+'.'+str(years[ye])+'.bio.mean.nc'
        f       = Dataset(ncfile, 'r')    
        par     = f.variables[par_id][ind,:]
        ncfile  = resultpath+runid+'.'+str(years[ye])+'.oce.diag.nc'
        f       = Dataset(ncfile, 'r')  
        mld     = f.variables[mld_id][ind,:]

     else:
        if ind ==4: dayind = (range(45,59))
        elif ind ==5: dayind = (range(60,74))
        elif ind ==6: dayind = (range(75,89))
        elif ind ==7: dayind = (range(90,105))
        elif ind ==8: dayind = (range(106,120))
        elif ind ==9: dayind = (range(121,135))
            
        ncfile  = resultpath+runid+'.'+str(years[ye])+'.bio.mean.nc'
        f       = Dataset(ncfile, 'r')    
        par     = f.variables[par_id][dayind,:].mean(axis=0)
        ncfile  = resultpath+runid+'.'+str(years[ye])+'.oce.diag.nc'
        f       = Dataset(ncfile, 'r')  
        mld     = f.variables[mld_id][dayind,:].mean(axis=0)
        
     depth_list = [-0,-10,-20]
        
     for i in range(len(depth_list)):

        print 'Load data at '+str(depth_list[i])+'m'
        level_data1, elem_no_nan1 = pf.get_data(par,mesh,depth_list[i])
        level_data[:,i]  = level_data1
        
     data_int=np.trapz(level_data,depth_list,axis=1)
     PAR[ye,:]=data_int
    data2=PAR.mean(axis=1)
    print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
    print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
    print 'Mean of surface: ',np.mean(data2[0:len(mesh.x2)])
    print 'Max and min: ',np.max(data2),np.min(data2)

    if export_csv == True:
        np.savetxt(outputpath+'PARmld_SEP_'+str(first_year)+'_'+str(last_year)+'_trend.csv', PAR, delimiter=";")
        np.savetxt(outputpath+'PARmld_SEP_'+str(first_year)+'_'+str(last_year)+'_mean.csv', data2, delimiter=";")
        print 'exporting done ...'

4
1990


NameError: name 'ind' is not defined

In [20]:
print level_data.shape

(753179, 3)


In [21]:
data_int=np.trapz(level_data,depth_list,axis=1)

In [22]:
data_int.shape

(753179,)

In [32]:
depth_list

array([], dtype=float64)